In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import GPy
from pandas import Series, DataFrame
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import norm
from IPython.core.pylabtools import figsize
from IPython.display import display

SAVE_FIG = True

if SAVE_FIG:
    from matplotlib import rcParams  
    rcParams['axes.labelsize'] = 9  
    rcParams['xtick.labelsize'] = 9  
    rcParams['ytick.labelsize'] = 9  
    rcParams['legend.fontsize'] = 9 
    default_fig_w = 8
    default_fig_h = 5
else:
    default_fig_w = 16
    default_fig_h = 10

%matplotlib inline
figsize(default_fig_w, default_fig_h)

# Load and adjust the data

In [3]:
df_train = pd.read_csv('../data/TableS2_training_hmdb.csv', keep_default_na=False)
df_train['RRF_Measured'] = df_train['RRF_Measured'] * 1e-2
df_train['RRF_Predicted'] = df_train['RRF_Predicted'] * 1e-2

df_test = pd.read_csv('../data/TableS3_training_hmdb.csv', keep_default_na=False)
df_test['RRF_Measured'] = df_test['RRF_Measured'] * 1e-2
df_test['RRF_Predicted'] = df_test['RRF_Predicted'] * 1e-2

features = [
    'JCHEM_ACIDIC_PKA',
    'JCHEM_LOGP',
    'JCHEM_POLARIZABILITY',
    'JCHEM_POLAR_SURFACE_AREA',
]

target = ['RRF_Measured']

# Data preparation
- Exclude certain metabolites
- Select training and testing data

In [4]:
# Training Data
X_train = df_train[features]
X_train = X_train[df_train['Annotation'] == '']

y_train = df_train[target]
y_train = y_train[df_train['Annotation'] == '']

# Testing Data
X_test = df_test[features]

y_test = df_test[target]

print("Number of metabolites (training):", len(X_train))
print("Number of metabolites (testing):", len(X_test))

Number of metabolites (training): 34
Number of metabolites (testing): 10


# Fit Gaussian Process

In [5]:
model = GPy.models.GPRegression(X_train, y_train, normalizer=True)
model.optimize()

display(model)

# Make predictions

In [17]:
mu, var = model.predict(X_test.as_matrix())
s = np.sqrt(var)

likelihood = np.sum([norm.logpdf(t[0], loc=t[1], scale=t[2]) for t in zip(y_test[target].values, mu.flatten(), s.flatten())])
print('Test Log-likelihood', likelihood)

Test Log-likelihood 34.8901990295


In [ ]:
data_test = pd.concat(
    [
        df_test, 
        DataFrame({'RRF_Predicted_New': mu.flatten(), 'RRF_Predicted_Std_New': s.flatten()}),
    ], axis=1)

data_test[['Name', 'RRF_Measured', 'RRF_Predicted', 'RRF_Predicted_New']]

# Comparison of model metrics

In [ ]:
y_true = data_test['RRF_Measured']
y_pred = data_test['RRF_Predicted']
y_pred_new = data_test['RRF_Predicted_New']

In [ ]:
figsize(1.2 * default_fig_w, default_fig_h)

ax = plt.subplot(1, 2, 1)
plt.xlabel('RRF Measured')
plt.ylabel('RRF Predicted (Reported)')
plt.scatter(y_true, y_pred)
plt.errorbar(y_true, y_pred, fmt='o', elinewidth=1, capthick=1, capsize=3, mew=0)

xlim = plt.xlim()
plt.plot([xlim[0], xlim[1]], [xlim[0], xlim[1]], 'r--', linewidth=.3)
plt.xlim(*xlim)

for x, y, label in zip(y_true, y_pred, df_test.ix[X_test.index]['Short Name']):
    ann = ax.annotate(label, (x + 0.002, y), alpha=0.8)

ylim = plt.ylim()

plt.title('Reported Linear Model (Chalcraft et al.) $R^2$={:.4f}'.format(r2_score(y_true, y_pred)))
#---

ax = plt.subplot(1, 2, 2)
plt.xlabel('RRF Measured')
plt.ylabel('RRF Predicted (New)')
plt.scatter(y_true, y_pred_new)
plt.errorbar(y_true, y_pred_new, yerr=np.sqrt(var), fmt='o', elinewidth=1, capthick=1, capsize=3)

plt.plot([xlim[0], xlim[1]], [xlim[0], xlim[1]], 'r--', linewidth=.3)
plt.xlim(*xlim)
plt.ylim(*ylim)

for x, y, label in zip(y_true, y_pred_new, df_test.ix[X_test.index]['Short Name']):
    ann = ax.annotate(label, (x + 0.002, y), alpha=0.8)

plt.grid(True)
plt.title('GPR Model $R^2$={:.4f}'.format(r2_score(y_true, y_pred_new)))

plt.tight_layout(pad=0.1)

if SAVE_FIG:
    plt.savefig('../figures/hmdb_linear_and_gp_regression.pdf')

plt.show()

## Metrics of the model reported in the paper

In [ ]:
print("mean_squared_error", mean_squared_error(y_true, y_pred))
print("r2_score", r2_score(y_true, y_pred))

## Metrics of our new models

In [ ]:
print("mean_squared_error", mean_squared_error(y_true, y_pred_new))
print("r2_score", r2_score(y_true, y_pred_new))
print("Log Likeliehood", np.sum([norm.logpdf(t[0], loc=t[1], scale=t[2]) for t in zip(y_true, y_pred_new, data_test['RRF_Predicted_Std_New'])]))